In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=51d1f3764ddbc79c1b62de91ea14398a5b22584101dfb4f1d36b662e2c867871
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import re
import fastai
from sentence_transformers import SentenceTransformer, util

# Load the tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained("skt/kogpt2-base-v2")
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# Read the CSV dataset
df = pd.read_csv('./sejongAQL.csv')

# Extract questions and answers from the DataFrame
questions = df['Q'].tolist()
answers = df['A'].tolist()

# Define the tokenizer for FastAI
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encodes(self, x):
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))

    def decodes(self, x):
        return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

# Load the pre-trained sentence transformer model
sentence_transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the answer generator function
def generate_answer(question):
    # Encode the input question
    question_embedding = sentence_transformer_model.encode([question])

    # Compute cosine similarities between the question and all stored questions
    similarities = util.pytorch_cos_sim(question_embedding, question_embeddings).flatten()

    # Find the index of the most similar question
    most_similar_idx = similarities.argmax().item()

    # Retrieve the corresponding answer
    answer = answers[most_similar_idx]

    return answer

# Encode all stored questions
question_embeddings = sentence_transformer_model.encode(questions)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


장영실의 물시계는 조선시대에 과학 기술 발전과 시간 관리의 필요성에 따라 개발되었습니다. 정확한 시간을 알고자 함과 함께 농경, 군사, 예술 등 다양한 분야에서 활용하기 위해 개발되었습니다.


In [ ]:

# Example usage
question = "세종대왕은 언제 왕이 되었나요?"
answer = generate_answer(question)
print(answer)


두 달 뒤인 1418년 8월 10일 태종의 선위를 이어받아 세자 충녕대군이 왕위에 올랐으니 이 사람이 세종이다.
